In [225]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np


In [2]:
BASE_URL = "https://aoe2.net/api/strings?game=aoe2de&language=en"

In [3]:
#res = req.get(BASE_URL)
#res

In [4]:
#res.json().keys()

In [5]:
#res.json()["civ"]

In [6]:
#res.json()["civ"][0]

In [7]:
#res.json()["civ"][0]["string"]
#está aqui el nombre de las civilizaciones que quiero

In [8]:
#data = pd.DataFrame(columns = ["civ_id", "civs"])
#creo un data frame para introducir las civilizaciones y su nombre

In [9]:
#data.head()

In [10]:
#data.civs = [civ["string"] for civ in res.json()["civ"]]
#lleno civs con las civilizaciones

In [11]:
#data.head()

In [12]:
#data["civ_id"] = [i for i in range(len(data.civs))]

In [13]:
#data.head(2)

In [14]:
#data.tail(2)

In [15]:
#hecho
#ahora vamos a sacar la informacion de todas la unidades a traves de una pagina web escrapeando:
#https://www.aoe2database.com/unit/175/1/en siendo 175 la ultima y 1 la primera
#vamos a sacar los siguientes atributos de cada elemento:
#nombre de la unidad, training_building, age, class, required rechnology, upgraded from, next updrage
#hit_points, attack, melee_armor, pierce_armor, range, minimum_range, line_of_sight, reolad_time
#speed, blast_radius, attack_delay, accuracy, number_projectiles, projectile_speed
#garrison_capacity, population, work_rate, heal_rate
#cost_wood, cost_gold, cost_food, cost_rock, training_time

In [76]:
url = "https://www.aoe2database.com/unit/175/1/en"

PATH = ChromeDriverManager().install() 
driver=webdriver.Chrome(PATH)  

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/1014459898.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


In [77]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [78]:
driver = ChromeDriverManager().install()  #remember substitute this for your driver path
driver = webdriver.Chrome(driver,options = opciones)

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3366603859.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver,options = opciones)


In [19]:
driver.get(url)
#todo = driver.find_elements(By.CLASS_NAME, "h3.lh-condensed")
#aqui tiene que esperar
time.sleep(4)

In [20]:
nombre_unidad = driver.find_elements(By.CLASS_NAME, "title")

In [21]:
nombre_unidad[0].text

'Elite Chakram Thrower'

In [22]:
training_building = driver.find_elements(By.CLASS_NAME, "entity-name")

In [23]:
training_building[5].text #si voy iterando sobre los [], puedo escrapear mas cosas necesarias, de 0 a 5

'None'

In [24]:
 for i in range(6):
        print (training_building[i].text)

Castle
Imperial Age
Infantry
Elite Chakram Thrower
Chakram Thrower
None


In [25]:
#despues de estas pruebas, he podido escrapear algunas partes, vamos a centrarnos en el todo

In [26]:
#value-text
unit_stats = driver.find_elements(By.CLASS_NAME, "value-text")

In [27]:
unit_stats[17].text #con esto de 0 a 17 consigo lo que quiero

'0'

In [28]:
#//*[@id="ROOT-2521314"]/vaadin-app-layout/div[2]/div/div[2]/vaadin-tabs/vaadin-tab[6]

In [29]:
#//*[@id="ROOT-2521314"]/vaadin-app-layout/div[2]/div/div[2]/vaadin-tabs//div[3]
#nexts = driver.find_element(By.CLASS_NAME, 'forward-button')
#nexts.click()
#el contenido en el que tengo que clickar de momento lo dejo en standby.
#habria que hacer click por coordenada, no por botón

In [30]:
nombrecolumnas = ["id_uni", "name_uni", "training_building", "age", "class", "required_rechnology", "upgraded_from",
                  "next_updrage", "hit_points", "attack", "melee_armor", "pierce_armor", "range",
                  "minimum_range", "line_of_sight", "reolad_time", "speed", "blast_radius",
                  "attack_delay", "accuracy", "number_projectiles", "projectile_speed",
                 "garrison_capacity", "population", "work_rate", "heal_rate",
                 "cost_wood", "cost_gold", "cost_food", "training_time"]
unidades = pd.DataFrame(columns=nombrecolumnas)
unidades.head(2)
#creado el dataframe donde rellenar cuando scrapeemos

,id_uni,name_uni,training_building,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,...,number_projectiles,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time


In [31]:
unidades

,id_uni,name_uni,training_building,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,...,number_projectiles,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time


In [34]:
cont = 0
while cont < 175:
    contador = 0
    unidades = unidades.append({},ignore_index = True)
    url = f"https://www.aoe2database.com/unit/{cont+1}/1/en"
    driver.get(url)
    time.sleep(10)

    nombre_unidad = driver.find_elements(By.CLASS_NAME, "title")
    #nombre_unidad = driver.find_elements(By.CLASS_NAME, "title")
    unidades.id_uni[cont] = cont
    unidades.name_uni[cont] = nombre_unidad[0].text
    training_building = driver.find_elements(By.CLASS_NAME, "entity-name")
    unit_stats = driver.find_elements(By.CLASS_NAME, "value-text")
    for i in range(6):
        unidades.iloc[cont][i+2] = training_building[i].text
    for i in range(18):
        unidades.iloc[cont][i+8] = unit_stats[i].text
    resources1 = driver.find_elements(By.CLASS_NAME, "res-icon")
    resources = driver.find_elements(By.CLASS_NAME, "res-value")
    num_resources = len(resources)
    while contador < (num_resources-1):
        src = resources1[contador].get_attribute("src")
        if "wood" in src:
            unidades.cost_wood[cont] = resources[contador].text
        if "gold" in src:    
            unidades.cost_gold[cont] = resources[contador].text
        if "food" in src:
            unidades.cost_food[cont] = resources[contador].text
        contador += 1
    unidades.training_time[cont] = resources[num_resources - 1].text
    cont += 1
#con todo esto obtenemos la informacion necesaria de la pagina web que necesitamos
#Actualización. He tirado la web, no puedo obtener la información
#Actualización 2: ha vuelto a la vida la web

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

...

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unidades = unidades.append({},ignore_index = True)
/var/folders/bw/khd8ptmn3_9fkbzj7_596v7c0000gn/T/ipykernel_5204/3156272067.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in

In [35]:
unidades.tail()

,id_uni,name_uni,training_building,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,...,number_projectiles,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time
170,170,Siege Elephant,Siege Workshop,Imperial Age,Cavalry,Siege Elephant,Armored Elephant,None,250 (270),4 (+4),...,-,-,-,1,-,0,NaN,95,130,36
171,171,Elite Ratha (Ranged),Castle,Imperial Age,Cavalry Archer,Elite Ratha,Ratha (Ranged),None,115 (135),6 (+4),...,1,7,-,1,-,0,60,60,NaN,18 (13.53)
172,172,Elite Ratha (Melee),Castle,Imperial Age,Cavalry,Elite Ratha,Ratha (Melee),None,115 (135),12 (+4),...,-,-,-,1,-,0,60,60,NaN,18 (13.53)
173,173,Elite Urumi Swordsman,Castle,Imperial Age,Infantry,Elite Urumi Swordsman,Urumi Swordsman,None,65,10 (+4),...,-,-,-,1,-,0,NaN,20,65,9 (6.77)
174,174,Elite Chakram Thrower,Castle,Imperial Age,Infantry,Elite Chakram Thrower,Chakram Thrower,None,50,4 (+2),...,1,6,-,1,-,0,NaN,30,65 (49),15 (11.28)


In [36]:
unidades.head(5)

,id_uni,name_uni,training_building,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,...,number_projectiles,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time
0,0,Villager,Town Center,Dark Age,Civilian,None,None,None,25 (40),3,...,-,-,-,1,-,0,NaN,NaN,50,25
1,1,Fishing Ship,Dock,Dark Age,Fish Boat,None,None,None,60,-,...,-,-,-,1,0.28,0,75,NaN,NaN,40
2,2,Transport Ship,Dock,Dark Age,Transport Boat,None,None,None,100,-,...,-,-,5,1,-,0,125,NaN,NaN,46
3,3,Militia,Barracks,Dark Age,Infantry,None,None,Man-at-Arms,40,4,...,-,-,-,1,-,0,NaN,20,60,21 (18.9)
4,4,Trade Cog,Dock,Feudal Age,Trade Boat,None,None,None,80,-,...,-,-,-,1,0.3,0,100,50,NaN,36


In [37]:
unidades.to_csv("tabla_unidades.csv", index = False)

In [38]:
unidades["cost_wood"].fillna(0, inplace = True)
unidades["cost_gold"].fillna(0, inplace = True)
unidades["cost_food"].fillna(0, inplace = True)

In [ ]:
#training_time


In [41]:
unidades['training_time'] = unidades['training_time'].str.split().str[0]

In [42]:
unidades.head(5)

,id_uni,name_uni,training_building,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,...,number_projectiles,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time
0,0,Villager,Town Center,Dark Age,Civilian,None,None,None,25 (40),3,...,-,-,-,1,-,0,0,0,50,25
1,1,Fishing Ship,Dock,Dark Age,Fish Boat,None,None,None,60,-,...,-,-,-,1,0.28,0,75,0,0,40
2,2,Transport Ship,Dock,Dark Age,Transport Boat,None,None,None,100,-,...,-,-,5,1,-,0,125,0,0,46
3,3,Militia,Barracks,Dark Age,Infantry,None,None,Man-at-Arms,40,4,...,-,-,-,1,-,0,0,20,60,21
4,4,Trade Cog,Dock,Feudal Age,Trade Boat,None,None,None,80,-,...,-,-,-,1,0.3,0,100,50,0,36


In [43]:
#training_building
unidades.training_building.unique()

array(['Town Center', 'Dock', 'Barracks', 'Market', 'Archery Range',
       'Stable', 'Monastery', 'Siege Workshop', 'Castle', 'Donjon'],
      dtype=object)

In [79]:
url = "https://ageofempires.fandom.com/wiki/Units_(Age_of_Empires_II)"
#de aquí voy a extraer los nombres de los edificios para compararlos con los que ya tenemos

In [108]:
driver.get(url)


'MÁS INFORMACIÓNACEPTAR'

In [112]:
cockies = driver.find_elements(By.XPATH, '/html/body/div[6]/div/div/div[2]/div[2]')[0]


In [113]:
cockies.click()

In [114]:
#toctext
edificaciones = driver.find_elements(By.CLASS_NAME, "toctext")

In [118]:
edificaciones[27].text #del 17 al 27
#exploro donde están los nombres de los edificios

'Donjon'

In [119]:
edificios = [edificaciones[i].text for i in range(17,28)]

In [120]:
edificios
#cojo los edificios

['Town Center',
 'Barracks',
 'Archery Range',
 'Stable',
 'Siege Workshop',
 'Monastery',
 'Market',
 'Castle',
 'Dock',
 'Krepost',
 'Donjon']

In [121]:
edificios = pd.DataFrame(edificios)

In [123]:
edificios["id_edif"] = [i for i, row in enumerate(edificios.itertuples())]

In [126]:
edificios.rename(columns={edificios.columns[0]: 'nombre_edificios'}, inplace=True)

In [132]:
edificios

,nombre_edificios,id_edif
0,Town Center,0
1,Barracks,1
2,Archery Range,2
3,Stable,3
4,Siege Workshop,4
5,Monastery,5
6,Market,6
7,Castle,7
8,Dock,8
9,Krepost,9


In [125]:
unidades.training_building.unique()

array(['Town Center', 'Dock', 'Barracks', 'Market', 'Archery Range',
       'Stable', 'Monastery', 'Siege Workshop', 'Castle', 'Donjon'],
      dtype=object)

In [131]:
comparacion = edificios.nombre_edificios.isin(unidades.training_building.unique())
comparacion

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9     False
10     True
Name: nombre_edificios, dtype: bool

In [134]:
#se me ha olvidado hacer copia, la hago ahora
edificios_copy = edificios.copy()
unidades_copy = unidades.copy()

In [136]:
#como una de las unidades se puede entrenar tanto en castle como en krepost, eliminamos la fila de krepost
edificios = edificios[edificios.nombre_edificios != "Krepost"]

In [137]:
edificios

,nombre_edificios,id_edif
0,Town Center,0
1,Barracks,1
2,Archery Range,2
3,Stable,3
4,Siege Workshop,4
5,Monastery,5
6,Market,6
7,Castle,7
8,Dock,8
10,Donjon,10


In [138]:
edificios = edificios.reset_index(drop=True)

In [140]:
edificios["id_edif"] = [i for i, row in enumerate(edificios.itertuples())]

In [141]:
edificios

,nombre_edificios,id_edif
0,Town Center,0
1,Barracks,1
2,Archery Range,2
3,Stable,3
4,Siege Workshop,4
5,Monastery,5
6,Market,6
7,Castle,7
8,Dock,8
9,Donjon,9


In [ ]:
#training_building
#edificios.rename(columns={edificios.columns[0]: 'nombre_edificios'}, inplace=True)


In [146]:
unidades.rename(columns={'training_building': 'nombre_edificios'}, inplace=True)

In [147]:
unidades.head()

,id_uni,name_uni,nombre_edificios,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,...,number_projectiles,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time
0,0,Villager,Town Center,Dark Age,Civilian,None,None,None,25 (40),3,...,-,-,-,1,-,0,0,0,50,25
1,1,Fishing Ship,Dock,Dark Age,Fish Boat,None,None,None,60,-,...,-,-,-,1,0.28,0,75,0,0,40
2,2,Transport Ship,Dock,Dark Age,Transport Boat,None,None,None,100,-,...,-,-,5,1,-,0,125,0,0,46
3,3,Militia,Barracks,Dark Age,Infantry,None,None,Man-at-Arms,40,4,...,-,-,-,1,-,0,0,20,60,21
4,4,Trade Cog,Dock,Feudal Age,Trade Boat,None,None,None,80,-,...,-,-,-,1,0.3,0,100,50,0,36


In [149]:
#old_hdd= old_hdd.merge(actor, on= 'complete', how='left')
unidades = unidades.merge(edificios, on ="nombre_edificios", how= "left")
unidades.head()

,id_uni,name_uni,nombre_edificios,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,...,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time,id_edif
0,0,Villager,Town Center,Dark Age,Civilian,None,None,None,25 (40),3,...,-,-,1,-,0,0,0,50,25,0
1,1,Fishing Ship,Dock,Dark Age,Fish Boat,None,None,None,60,-,...,-,-,1,0.28,0,75,0,0,40,8
2,2,Transport Ship,Dock,Dark Age,Transport Boat,None,None,None,100,-,...,-,5,1,-,0,125,0,0,46,8
3,3,Militia,Barracks,Dark Age,Infantry,None,None,Man-at-Arms,40,4,...,-,-,1,-,0,0,20,60,21,1
4,4,Trade Cog,Dock,Feudal Age,Trade Boat,None,None,None,80,-,...,-,-,1,0.3,0,100,50,0,36,8


In [150]:
unidades.columns

Index(['id_uni', 'name_uni', 'nombre_edificios', 'age', 'class',
       'required_rechnology', 'upgraded_from', 'next_updrage', 'hit_points',
       'attack', 'melee_armor', 'pierce_armor', 'range', 'minimum_range',
       'line_of_sight', 'reolad_time', 'speed', 'blast_radius', 'attack_delay',
       'accuracy', 'number_projectiles', 'projectile_speed',
       'garrison_capacity', 'population', 'work_rate', 'heal_rate',
       'cost_wood', 'cost_gold', 'cost_food', 'training_time', 'id_edif'],
      dtype='object')

In [151]:
unidades = unidades.drop("nombre_edificios", axis=1)

In [153]:
unidades.columns

Index(['id_uni', 'name_uni', 'age', 'class', 'required_rechnology',
       'upgraded_from', 'next_updrage', 'hit_points', 'attack', 'melee_armor',
       'pierce_armor', 'range', 'minimum_range', 'line_of_sight',
       'reolad_time', 'speed', 'blast_radius', 'attack_delay', 'accuracy',
       'number_projectiles', 'projectile_speed', 'garrison_capacity',
       'population', 'work_rate', 'heal_rate', 'cost_wood', 'cost_gold',
       'cost_food', 'training_time', 'id_edif'],
      dtype='object')

In [154]:
#con esto he sustituido el nombre de los edificios por su id en la tabla edificios
#para que sea más fácil establecer la relación entre ellas en la database


,nombre_edificios,id_edif
0,Town Center,0
1,Barracks,1
2,Archery Range,2
3,Stable,3
4,Siege Workshop,4


In [156]:
#df['columna_especifica'].replace("-", "0", inplace=True)
unidades.replace("-", 0, inplace = True)
unidades.head()

,id_uni,name_uni,age,class,required_rechnology,upgraded_from,next_updrage,hit_points,attack,melee_armor,...,projectile_speed,garrison_capacity,population,work_rate,heal_rate,cost_wood,cost_gold,cost_food,training_time,id_edif
0,0,Villager,Dark Age,Civilian,None,None,None,25 (40),3,0 (+1),...,0,0,1,0,0,0,0,50,25,0
1,1,Fishing Ship,Dark Age,Fish Boat,None,None,None,60,0,0,...,0,0,1,0.28,0,75,0,0,40,8
2,2,Transport Ship,Dark Age,Transport Boat,None,None,None,100,0,4,...,0,5,1,0,0,125,0,0,46,8
3,3,Militia,Dark Age,Infantry,None,None,Man-at-Arms,40,4,0,...,0,0,1,0,0,0,20,60,21,1
4,4,Trade Cog,Feudal Age,Trade Boat,None,None,None,80,0,0,...,0,0,1,0.3,0,100,50,0,36,8


In [157]:
unidades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 174
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_uni               175 non-null    int64 
 1   name_uni             175 non-null    object
 2   age                  175 non-null    object
 3   class                175 non-null    object
 4   required_rechnology  175 non-null    object
 5   upgraded_from        175 non-null    object
 6   next_updrage         175 non-null    object
 7   hit_points           175 non-null    object
 8   attack               175 non-null    object
 9   melee_armor          175 non-null    object
 10  pierce_armor         175 non-null    object
 11  range                175 non-null    object
 12  minimum_range        175 non-null    object
 13  line_of_sight        175 non-null    object
 14  reolad_time          175 non-null    object
 15  speed                175 non-null    object
 16  blast_ra

In [161]:
unidades.age.unique() #vamos a crear otra tabla con esto y asignarle un id

array(['Dark Age', 'Feudal Age', 'Castle Age', 'Imperial Age'],
      dtype=object)

In [163]:
unidades["class"].unique() #la clase conquistador habria que hacer algo con ella de cara a otro proyecto

array(['Civilian', 'Fish Boat', 'Transport Boat', 'Infantry',
       'Trade Boat', 'Warship', 'Trade Cart', 'Archer', 'Cavalry', 'Monk',
       'Cavalry Archer', 'Siege Weapon', 'Petard', 'Conquistador',
       'Hand Cannoneer'], dtype=object)

In [168]:
pd.options.display.max_rows = 175
unidades.training_time.head()

0    25
1    40
2    46
3    21
4    36
Name: training_time, dtype: int64

In [166]:
unidades.training_time = unidades.training_time.astype(int) #para convertirlo a int

In [172]:
#unidades['training_time'] = unidades['training_time'].str.split().str[0]
unidades["cost_food"] = unidades["cost_food"].str.split().str[0]

In [175]:
unidades["cost_wood"] = unidades["cost_wood"].str.split().str[0]
unidades["cost_gold"] = unidades["cost_gold"].str.split().str[0]

In [176]:
unidades["cost_wood"].fillna(0, inplace = True)
unidades["cost_gold"].fillna(0, inplace = True)
unidades["cost_food"].fillna(0, inplace = True)

In [177]:
unidades["cost_wood"] = unidades["cost_wood"].astype(int)
unidades["cost_food"] = unidades["cost_food"].astype(int)
unidades["cost_gold"] = unidades["cost_gold"].astype(int)

In [178]:
unidades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 174
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_uni               175 non-null    int64 
 1   name_uni             175 non-null    object
 2   age                  175 non-null    object
 3   class                175 non-null    object
 4   required_rechnology  175 non-null    object
 5   upgraded_from        175 non-null    object
 6   next_updrage         175 non-null    object
 7   hit_points           175 non-null    object
 8   attack               175 non-null    object
 9   melee_armor          175 non-null    object
 10  pierce_armor         175 non-null    object
 11  range                175 non-null    object
 12  minimum_range        175 non-null    object
 13  line_of_sight        175 non-null    object
 14  reolad_time          175 non-null    object
 15  speed                175 non-null    object
 16  blast_ra

In [182]:

unidades["population"] = unidades["population"].astype(float)#lo paso a float porque hay valores de 0.5
unidades.population.head() #tengo algunas dudas sobre si están correctos los valores de esto
#no estoy seguro de si no hay alguna unidad que tenga una population mayor de uno, de momento lo dejamos asi


0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: population, dtype: float64

In [184]:
#unidades["cost_gold"] = unidades["cost_gold"].str.split().str[0]
#con esto nos quedamos con los numeros base sin los modificadores
#los modificadores ya serán dados por otras cosas en un futuro proyecto
unidades["hit_points"] = unidades["hit_points"].str.split().str[0]
unidades["attack"] = unidades["attack"].str.split().str[0]
unidades["melee_armor"] = unidades["melee_armor"].str.split().str[0]
unidades["pierce_armor"] = unidades["pierce_armor"].str.split().str[0]
unidades["range"] = unidades["range"].str.split().str[0]
unidades["minimum_range"] = unidades["minimum_range"].str.split().str[0]
unidades["line_of_sight"] = unidades["line_of_sight"].str.split().str[0]
unidades["reolad_time"] = unidades["reolad_time"].str.split().str[0]
unidades["speed"] = unidades["speed"].str.split().str[0]
unidades["blast_radius"] = unidades["blast_radius"].str.split().str[0]
unidades["attack_delay"] = unidades["attack_delay"].str.split().str[0]
unidades["accuracy"] = unidades["accuracy"].str.split().str[0]
unidades["number_projectiles"] = unidades["number_projectiles"].str.split().str[0]
unidades["projectile_speed"] = unidades["projectile_speed"].str.split().str[0]    
unidades["garrison_capacity"] = unidades["garrison_capacity"].str.split().str[0]    
unidades["work_rate"] = unidades["work_rate"].str.split().str[0]  
unidades["heal_rate"] = unidades["heal_rate"].str.split().str[0]  

In [186]:
unidades["hit_points"].fillna(0, inplace = True)
unidades["attack"].fillna(0, inplace = True)
unidades["melee_armor"].fillna(0, inplace = True)
unidades["pierce_armor"].fillna(0, inplace = True)
unidades["range"].fillna(0, inplace = True)
unidades["minimum_range"].fillna(0, inplace = True)
unidades["line_of_sight"].fillna(0, inplace = True)
unidades["reolad_time"].fillna(0, inplace = True)
unidades["speed"].fillna(0, inplace = True)
unidades["blast_radius"].fillna(0, inplace = True)
unidades["attack_delay"].fillna(0, inplace = True)
unidades["accuracy"].fillna(0, inplace = True)
unidades["number_projectiles"].fillna(0, inplace = True)
unidades["projectile_speed"].fillna(0, inplace = True) 
unidades["garrison_capacity"].fillna(0, inplace = True) 
unidades["work_rate"].fillna(0, inplace = True)
unidades["heal_rate"].fillna(0, inplace = True)

In [195]:
unidades["hit_points"] = unidades["hit_points"].astype(int)
unidades["attack"] = unidades["attack"].astype(int)
unidades["melee_armor"] = unidades["melee_armor"].astype(int)
unidades["pierce_armor"] = unidades["pierce_armor"].astype(int)


In [216]:
unidades["range"] = unidades["range"].astype(float)
unidades["minimum_range"] = unidades["minimum_range"].astype(int)
unidades["line_of_sight"] = unidades["line_of_sight"].astype(int)
unidades["reolad_time"] = unidades["reolad_time"].astype(float)
unidades["speed"] = unidades["speed"].astype(float)
unidades["blast_radius"] = unidades["blast_radius"].astype(float)
unidades["attack_delay"] = unidades["attack_delay"].astype(float)
unidades["number_projectiles"] = unidades["number_projectiles"].astype(int)
unidades["projectile_speed"] = unidades["projectile_speed"].astype(float)   
unidades["garrison_capacity"] = unidades["garrison_capacity"].astype(int)   
unidades["work_rate"] = unidades["work_rate"].astype(float) 
unidades["heal_rate"] = unidades["heal_rate"].astype(float)

In [217]:
unidades["accuracy"] = unidades["accuracy"].str.replace("%", "")

In [219]:
unidades["accuracy"].fillna(0, inplace = True)
unidades["accuracy"] = unidades["accuracy"].astype(int)

In [220]:
unidades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 174
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_uni               175 non-null    int64  
 1   name_uni             175 non-null    object 
 2   age                  175 non-null    object 
 3   class                175 non-null    object 
 4   required_rechnology  175 non-null    object 
 5   upgraded_from        175 non-null    object 
 6   next_updrage         175 non-null    object 
 7   hit_points           175 non-null    int64  
 8   attack               175 non-null    int64  
 9   melee_armor          175 non-null    int64  
 10  pierce_armor         175 non-null    int64  
 11  range                175 non-null    float64
 12  minimum_range        175 non-null    int64  
 13  line_of_sight        175 non-null    int64  
 14  reolad_time          175 non-null    float64
 15  speed                175 non-null    flo

In [229]:
#df['columna_ejemplo'] = pd.to_numeric(df['columna_ejemplo'], downcast='integer')
#for c in airbnb.select_dtypes('integer'):
    #airbnb[c]=pd.to_numeric(airbnb[c], downcast='integer')
for c in  unidades.select_dtypes("integer"):
    #unidades[c] = pd.to_numeric(unidades[c], downcast = "integer")
    unidades[c] = unidades[c].astype(np.int16)
for c in unidades.select_dtypes("float"):
        unidades[c] = unidades[c].astype(np.float16)

In [230]:
unidades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 174
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_uni               175 non-null    int16  
 1   name_uni             175 non-null    object 
 2   age                  175 non-null    object 
 3   class                175 non-null    object 
 4   required_rechnology  175 non-null    object 
 5   upgraded_from        175 non-null    object 
 6   next_updrage         175 non-null    object 
 7   hit_points           175 non-null    int16  
 8   attack               175 non-null    int16  
 9   melee_armor          175 non-null    int16  
 10  pierce_armor         175 non-null    int16  
 11  range                175 non-null    float16
 12  minimum_range        175 non-null    int16  
 13  line_of_sight        175 non-null    int16  
 14  reolad_time          175 non-null    float16
 15  speed                175 non-null    flo